In [ ]:
import numpy as np

from weight import *
from utils import save, load
from library import *
from process_library_terms import *

import scipy.io
import matplotlib.pyplot as plt
mat = scipy.io.loadmat('KSres2.mat')
u = mat['uu']
dt = mat['dt'][0][0]
dx = mat['dx'][0][0]

fig, ax= plt.subplots(figsize=(6,6))
ax.imshow(u[:, :])

u = u[..., np.newaxis]

In [ ]:
uobs = Observable('u', 1)
print("u shape:", u.shape)

### define all parameters and "make library"
terms = generate_terms_to(5, observables=[uobs], max_observables=3)
terms0 = []
terms1 = []
for term in terms:
    if term.rank==0:
        terms0.append(term)
    else:
        terms1.append(term)

world_size = np.array(u.shape)
m = 6
qmax = 1
nweights = qmax**2
ndomains = 200
dom_width = 128
dom_time = 20
pad = 0

# fix random seed
np.random.seed(1)

data_dict = {'u': u}

dxs = [dx, dt]

scale_dict = find_scales(data_dict)

char_sizes0 = [get_char_size(term, scale_dict, dx, dt) for term in terms0]
char_sizes1 = [get_char_size(term, scale_dict, dx, dt) for term in terms1]

weights = []
dxs_weight = [(dom_width-1)/2*dx, (dom_time-1)/2*dt] # grid spacing used by weight function

#for domain in domains:
for q1 in range(qmax):
    for q2 in range(qmax):
        weights.append(Weight([m]*2, [q1, q2], [0, 0], dxs=dxs_weight))
#domains = domains*nweights

# make domains and "count" density
domains = make_domains([dom_width, dom_time], world_size, ndomains, pad)

Q0 = make_library(terms0, data_dict, weights, domains, 0, dxs)#, by_parts=False)
Q1 = make_library(terms1, data_dict, weights, domains, 1, dxs)#, by_parts=False)

In [ ]:
ks_terms = ['dt u_i', 'dj^2 u_i', 'dj^2 dk^2 u_i', 'dj u_j * u_i']
col_numbers = [find_term(terms1, name) for name in ks_terms]
Xi, lambd = regress(Q1, col_numbers)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {terms0[i]}. (Char. size: {char_sizes0[i]})")
print("Model residual:", lambd)